In [1]:
import os

import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error

from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, lit

from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

### Spark Configuration 

In [19]:
MAX_MEMORY = "5g"
spark = (
    SparkSession.builder.appName("movie-recommendation")
    .config("spark.executor.memory", MAX_MEMORY)
    .config("spark.driver.memory", MAX_MEMORY)
    .getOrCreate()
)

### 데이터 불러오기
- rating_df : 유저와 영화에 대한 평점 데이터셋
- movie_df : 영화에 대한 제목 데이터셋

In [3]:
DIR_PATH = os.path.join(os.getcwd(), 'data')
RATING_PATH = os.path.join(DIR_PATH, 'ratings.csv')
MOVIE_PATH = os.path.join(DIR_PATH, 'movies.csv')

rating_df = spark.read.csv(f"file:///{RATING_PATH}", inferSchema=True, header=True)
movie_df = spark.read.csv(f"file:///{MOVIE_PATH}", inferSchema=True, header=True)

In [4]:
rating_df.show(3)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
+------+-------+------+----------+
only showing top 3 rows



In [5]:
movie_df.show(3)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
+-------+--------------------+--------------------+
only showing top 3 rows



---

### 모델링을 위한 데이터 전처리
- 모델링에 사용하지 않을 timestamp 변수 제거

In [6]:
rating_df = rating_df.select(['userId', 'movieId', 'rating'])
rating_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



#### Train / Test Dataset Split

In [7]:
train_df, test_df = rating_df.randomSplit([0.8, 0.2], seed=13)

---

### ALS 추천 모델링

#### 모델 생성 및 학습

In [8]:
als = ALS(
    maxIter=10,
    regParam=0.01,
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating',
    nonnegative=True,
    coldStartStrategy='drop'
)

model = als.fit(train_df)

22/03/03 13:56:04 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/03/03 13:56:04 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


---

### 모델 평가

#### Test 데이터셋을 통한 예측 진행
- 예측값의 평균은 약 3.5로 실제값의 평균과 크게 다르지 않으며,
- 예측값의 범위는 0.0 ~ 10.42 이다.

In [9]:
prediction = model.transform(test_df)
prediction.show(5)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|    31|   6620|   1.5| 2.5002856|
|    76|   1959|   5.0| 3.6079454|
|   243|   1580|   3.0|   2.60493|
|   321|   3175|   3.0| 3.3042073|
|   321| 175197|   0.5| 1.6218803|
+------+-------+------+----------+
only showing top 5 rows



In [10]:
prediction.select(['rating', 'prediction']).describe().show()

+-------+------------------+------------------+
|summary|            rating|        prediction|
+-------+------------------+------------------+
|  count|           5001747|           5001747|
|   mean| 3.534025511486287| 3.458667017715034|
| stddev|1.0605557917302697|0.7129707775865479|
|    min|               0.5|               0.0|
|    max|               5.0|         12.817463|
+-------+------------------+------------------+



#### RMSE 지표를 통한 모델 평가
- 예측값과 실제값의 RMSE는 약 0.804으로, 평점에 대한 예측과 실제의 차이가 약 0.8점 정도 존재한다고 할 수 있다.
- 예측값의 범위를 0.0 ~ 5.0 로 스케일링한 뒤, 성능 평가를 한 결과, 0.803의 score를 가졌다.

In [11]:
evaluator = RegressionEvaluator(
    labelCol='rating',
    predictionCol='prediction',
    metricName='rmse'
)

rmse = evaluator.evaluate(prediction)
print(rmse)

0.8038765815985407


In [12]:
# 실제값과 예측값을 pandas dataframe으로 추출
datas = prediction.select(['rating', 'prediction']).toPandas()

# 예측값 중 5.0 이상인 값들을 5.0로 축소
datas['scaled_prediction'] = (
    datas['prediction']
    .map(lambda x: 5.0 if x >= 5.0 else x)
)

In [13]:
pred_y = datas.scaled_prediction.values
true_y = datas.rating

rmse = np.sqrt(mean_squared_error(true_y, pred_y))
rmse

0.8028263263037656

---

### 특정 유저에 대한 영화 추천 모델 생성
- 특정 유저의 ID를 인풋 데이터로 넣으면,
- 해당 유저가 본 영화를 제외하고, 그 외의 영화들을 ALS 모델의 인풋 데이터로 넣어 예측값이 높은 순으로 n개의 영화를 추천

In [14]:
# 전체 영화의 유니크한 ID값을 추출한 a라는 데이터프레임 생성
unique_movies = rating_df.select('movieId').distinct()
a = unique_movies.alias('a')


def recommendation_movies(user_id, model, n):
    # user_id가 시청한 영화를 추출한 뒤 b라는 데이터프레임에 저장
    watched_movies = rating_df.filter(rating_df['userId'] == user_id).select('movieId')
    b = watched_movies.alias('b')
    
    # 전체 영화 중 해당 유저가 본 영화를 제외한 영화 추출
    unwatched_movies = (
        a
        .join(b, a['movieId'] == b['movieId'], how='left')
        .where(col('b.movieId').isNull())
        .select('a.movieId').distinct()
    )
    
    # unwatched_movies 데이터프레임에 해당 유저의 ID를 삽입
    unwatched_movies = unwatched_movies.withColumn('userId', lit(int(user_id)))
    
    # 해당 유저가 안 본 영화 중 n개의 영화를 추천 : 예측값이 높은 순으로
    n_recommend_movies = (
        model
        .transform(unwatched_movies)
        .orderBy('prediction', ascending=False)
        .limit(n)
    )
    
    # 추천되는 영화 ID에 대응하는 영화 제목을 붙인 데이터프레임 생성
    n_recommend_movies = (
        n_recommend_movies
        .join(movie_df, n_recommend_movies['movieId'] == movie_df['movieId'])
        .orderBy('prediction', ascending=False)
        .select([
            'userId', 'a.movieId', 'title', 'genres', 'prediction'
        ])
    )
    
    return n_recommend_movies.toPandas()

In [15]:
user_id = 243

recommendation_movies(user_id, model, 10)

,userId,movieId,title,genres,prediction
0,243,152043,Leader (2010),Drama|Romance,8.985197
1,243,177657,Bullets for the Dead (2015),Horror|Western,8.795678
2,243,198657,Manikarnika (2019),Action|Drama,8.582273
3,243,155549,Borderline (1930),Drama,8.462073
4,243,185645,Stone Cold Steve Austin: The Bottom Line on th...,Documentary,8.232800
5,243,80719,Kanchenjungha (1962),Drama,7.962666
6,243,153550,Pyaar Ke Side Effects (2006),Comedy|Drama|Romance,7.943258
7,243,154860,Mother (2016),(no genres listed),7.849785
8,243,133327,Linda and Abilene (1969),Drama|Western,7.827180
9,243,86952,Son of Babylon (Syn Babilonu) (2009),Drama,7.599072


#### ALS method를 이용한 추천 모델 단순화

In [16]:
def recommendation_movies_simplify(user_id, model, n):
    users_df = spark.createDataFrame([user_id], IntegerType()).toDF('userId')
    
    recommendation_list = (
        model
        .recommendForUserSubset(users_df, 10)
        .collect()[0].recommendations
    )
    
    recommendation_df = spark.createDataFrame(recommendation_list)
    
    recommendation_df.createOrReplaceTempView("recommendations")
    movie_df.createOrReplaceTempView("movies")

    query = """
    SELECT M.movieId, M.title, M.genres, R.rating
    FROM movies M
    JOIN recommendations R
    ON M.movieId = R.movieId
    ORDER BY rating desc
    """
    
    recommended_movies = spark.sql(query)
    
    return recommended_movies.toPandas()

In [17]:
recommendation_movies_simplify(user_id, model, 10)

/Users/aeea/miniforge3/envs/spark_env/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


,movieId,title,genres,rating
0,152043,Leader (2010),Drama|Romance,8.985197
1,177657,Bullets for the Dead (2015),Horror|Western,8.795678
2,198657,Manikarnika (2019),Action|Drama,8.582273
3,155549,Borderline (1930),Drama,8.462073
4,185645,Stone Cold Steve Austin: The Bottom Line on th...,Documentary,8.232800
5,80719,Kanchenjungha (1962),Drama,7.962666
6,153550,Pyaar Ke Side Effects (2006),Comedy|Drama|Romance,7.943258
7,154860,Mother (2016),(no genres listed),7.849785
8,133327,Linda and Abilene (1969),Drama|Western,7.827180
9,86952,Son of Babylon (Syn Babilonu) (2009),Drama,7.599072


#### Spark Session 종료

In [18]:
spark.stop()